<a href="https://colab.research.google.com/github/Anchit-2002/Phising-Website-Detection-/blob/main/Model_Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import pickle
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
import gradio as gr

In [ ]:
# Load the trained model
with open('/content/rf model', 'rb') as f:
    model = pickle.load(f)

In [ ]:
def get_title_from_url(url):
    # Send a GET request to the URL
    response = requests.get(url)

    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract the title from the <title> tag
        title = soup.title.string if soup.title else None

        # Return the decoded title or a placeholder if it's None
        return title if title else "No Title Found"
    else:
        return "Failed to retrieve URL"


In [ ]:
# Function for checking empty or generic title
def check_empty_or_generic_title(title):

    generic_terms = ['home', 'index', 'default', 'welcome', 'about', 'contact', 'services', 'login', 'register']

    title = title.lower() if title else ''
    if not title or any(term in title for term in generic_terms):
        return 1
    return 0

def check_encoding_artifacts(content):
    non_ascii = re.findall(r'[^\x00-\x7F]', content)
    return len(non_ascii) > 0

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import string


# Load the trained model
with open('rf model','rb') as f:
    model = pickle.load(f)

# Function to preprocess the input URL
def preprocess_url(url):
    try:
        response = requests.get(url, timeout=10)
        content = response.text
        soup = BeautifulSoup(content, 'html.parser')
    except Exception:
        content = ""
        soup = None
    # Extract features
    features = {
        'URLLength': len(url),
        'DomainLength': len(url.split('/')[2]),
        'IsDomainIP': 1 if url.split('/')[2].replace('.', '').isdigit() else 0,
        'URLSimilarityIndex': 0.5,  # Placeholder value
        'CharContinuationRate': sum(
            1 for i in range(len(url) - 1) if url[i].isalnum() and url[i + 1].isalnum()
        ) / len(url) if len(url) > 1 else 0,
        'TLDLegitimateProb': 0.9,  # Placeholder value
        'URLCharProb': 0.8,  # Placeholder value
        'TLDLength': len(url.split('/')[2].split('.')[-1]),
        'NoOfSubDomain': len(url.split('/')[2].split('.')) - 1,
        'HasObfuscation': 1 if re.search(r'(0x|%[0-9A-Fa-f]{2}|[a-z]\d[a-z])', url) else 0,
        'NoOfObfuscatedChar': sum(1 for match in re.findall(r'(0x|%[0-9A-Fa-f]{2}|[a-z]\d[a-z])', url)),
        'ObfuscationRatio': sum(1 for c in url if c not in string.ascii_letters + string.digits) / len(url),
        'NoOfLettersInURL': sum(c.isalpha() for c in url),
        'LetterRatioInURL': sum(c.isalpha() for c in url) / len(url) if len(url) > 0 else 0,
        'NoOfDegitsInURL': sum(c.isdigit() for c in url),
        'DegitRatioInURL': sum(c.isdigit() for c in url) / len(url) if len(url) > 0 else 0,
        'NoOfEqualsInURL': url.count('='),
        'NoOfQMarkInURL': url.count('?'),
        'NoOfAmpersandInURL': url.count('&'),
        'NoOfOtherSpecialCharsInURL': sum(
            1 for c in url if c not in string.ascii_letters + string.digits + '&' + ':/?=#.'
        ),
        'SpacialCharRatioInURL': sum(
            1 for c in url if not c.isalnum()
        ) / len(url) if len(url) > 0 else 0,
        'IsHTTPS': 1 if url.startswith("https://") else 0,
        'LineOfCode': len(content.splitlines()) if content else 0,
        'LargestLineLength': max(len(line) for line in content.splitlines()) if content else 0,
        'HasTitle': 1 if soup and soup.title else 0,
        'DomainTitleMatchScore': 1 if soup and soup.title and url.split('/')[2] in soup.title.string else 0,
        'HasFavicon': 1 if soup and soup.find("link", rel="icon") else 0,
        'Robots': 1 if soup and soup.find("meta", attrs={"name": "robots"}) else 0,
        'IsResponsive': 1 if soup and soup.find("meta", attrs={"name": "viewport"}) else 0,
        'NoOfURLRedirect': len(response.history) if response else 0,
        'NoOfSelfRedirect': 1 if response and response.history and response.history[-1].url == url else 0,
        'HasDescription': 1 if soup and soup.find("meta", attrs={"name": "description"}) else 0,
        'NoOfPopup': content.count("window.open") if content else 0,
        'NoOfiFrame': len(soup.find_all("iframe")) if soup else 0,
        'HasExternalFormSubmit': 1 if soup and soup.find("form", action=lambda x: x and url.split('/')[2] not in x) else 0,
        'HasSocialNet': 1 if soup and any(network in content.lower() for network in ["facebook", "twitter", "linkedin"]) else 0,
        'HasSubmitButton': 1 if soup and soup.find("button", type="submit") else 0,
        'HasHiddenFields': 1 if soup and soup.find("input", type="hidden") else 0,
        'HasPasswordField': 1 if soup and soup.find("input", type="password") else 0,
        'Bank': 1 if re.search(r'\b(bank|banking)\b', content.lower()) else 0,
        'Pay': 1 if re.search(r'\b(pay|payment)\b', content.lower()) else 0,
        'Crypto': 1 if re.search(r'\b(crypto|cryptocurrency)\b', content.lower()) else 0,
        'HasCopyrightInfo': 1 if re.search(r'\b(copyright|©)\b', content.lower()) else 0,
        'NoOfImage': len(soup.find_all("img")) if soup else 0,
        'NoOfCSS': len(soup.find_all("link", rel="stylesheet")) if soup else 0,
        'NoOfJS': len(soup.find_all("script")) if soup else 0,
        'NoOfSelfRef': len(soup.find_all("a", href=lambda x: x and url.split('/')[2] in x)) if soup else 0,
        'NoOfEmptyRef': len(soup.find_all("a", href=lambda x: not x or x == "#")) if soup else 0,
        'NoOfExternalRef': len(soup.find_all("a", href=lambda x: x and url.split('/')[2] not in x)) if soup else 0,
        'EncodingArtifacts': 0,  # Placeholder: Replace with actual encoding check function
        'IsEmptyOrGenericTitle': 0,  # Placeholder: Replace with actual title check function
    }
    return features



In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

with open("pca", "rb") as f:
    pca = pickle.load(f)

scaler = StandardScaler()

def predict_url(url):
    try:
        # Preprocess the URL to extract features
        features = preprocess_url(url)
        features_array = np.array(list(features.values())).reshape(1, -1)

        # Standardize and apply PCA using the preloaded scaler and PCA
        features_scaled = scaler.fit_transform(features_array)  # Use preloaded scaler
        features_pca = pca.transform(features_scaled)       # Use preloaded PCA

        # Make a prediction using the trained model
        prediction = model.predict(features_pca)

        # Map the prediction to phishing or legitimate
        if prediction == 0:
            return "Phishing"
        if prediction ==1:
            return "Legitimate"


    except Exception as e:
        # Handle errors during preprocessing or prediction
        print(f"An error occurred: {e}")
        return None




In [ ]:
# predict_url('	http://rapidfsf.com')

In [ ]:
# Create the Gradio interface
iface = gr.Interface(
    fn= predict_url,
    inputs="text",
    outputs="text",
    title="Phishing URL Classifier",
    description="Enter a URL to check if it's phishing or legitimate."
)

# Launch the Gradio interface
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9dc7e438205d1c0a6e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
